In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [3]:
# import the required libraries
from czitools import metadata_tools as czimd
from czitools import misc_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
from pathlib import Path
import os
import requests
import ipywidgets as widgets
import glob

### Define Parameters for Data Loading

In [9]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [11]:
if not IN_COLAB:
    # choose local file
    fc = FileChooser()
    fc.default_path = INPUT_FOLDER
    fc.filter_pattern = '*.czi'
    display(fc)

elif IN_COLAB:
    # lislt files inside the folder om gdrive
    czifiles = glob.glob(os.path.join(INPUT_FOLDER, "*.czi"))
    wd = widgets.Select(
        options=czifiles,
        description='CZI Files:',
        layout={'width': 'max-content'}
    )
    display(wd)

FileChooser(path='E:\Github\czitools\data', filename='', title='', show_hidden=False, select_desc='Select', ch…

In [14]:
if not IN_COLAB:
    filepath = fc.selected
elif IN_COLAB:
    filepath = wd.value

print(f"Selected File: {filepath}")

Selected File: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small.czi


In [15]:
# get only specific metadata
czi_dimensions = czimd.CziDimensions(filepath)
print("SizeS: ", czi_dimensions.SizeS)
print("SizeT: ", czi_dimensions.SizeT)
print("SizeZ: ", czi_dimensions.SizeZ)
print("SizeC: ", czi_dimensions.SizeC)
print("SizeY: ", czi_dimensions.SizeY)
print("SizeX: ", czi_dimensions.SizeX)

2024-02-11 14:16:19,465 -  INFO - Reading Dimensions from CZI image data.


SizeS:  None
SizeT:  10
SizeZ:  15
SizeC:  2
SizeY:  256
SizeX:  256


In [16]:
# and get more info
czi_scaling = czimd.CziScaling(filepath)
czi_channels = czimd.CziChannelInfo(filepath)
czi_bbox = czimd.CziBoundingBox(filepath)
czi_objectives = czimd.CziObjectives(filepath)
czi_detectors = czimd.CziDetector(filepath)
czi_microscope = czimd.CziMicroscope(filepath)
czi_sample = czimd.CziSampleInfo(filepath)

2024-02-11 14:17:00,463 -  INFO - Reading Scaling from CZI image data.
2024-02-11 14:17:00,715 -  INFO - Reading Channel Information from CZI image data.
2024-02-11 14:17:00,925 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-11 14:17:00,926 -  INFO - Reading Objective Information from CZI image data.
2024-02-11 14:17:01,133 -  INFO - Reading Detector Information from CZI image data.
2024-02-11 14:17:01,322 -  INFO - Reading Microscope Information from CZI image data.
2024-02-11 14:17:01,516 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-11 14:17:01,735 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 14:17:01,735 -  INFO - No Scene or Well information found. Try to read XY Stage Coordinates from subblocks.
2024-02-11 14:17:01,940 -  INFO - Reading Dimensions from CZI image data.


In [17]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(filepath)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc_tools.md2dataframe(mdict)

# create a ipywdiget to show the dataframe with the metadata
wd1 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd1:
    display(HTML(mdframe.to_html()))
display(widgets.VBox(children=[wd1]))

2024-02-11 14:17:08,873 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 14:17:08,876 -  INFO - Reading BoundingBoxes from CZI image data.
2024-02-11 14:17:08,878 -  INFO - Reading Channel Information from CZI image data.
2024-02-11 14:17:08,878 -  INFO - Reading Scaling from CZI image data.
2024-02-11 14:17:08,879 -  INFO - Reading Objective Information from CZI image data.
2024-02-11 14:17:08,880 -  INFO - Reading Detector Information from CZI image data.
2024-02-11 14:17:08,881 -  INFO - Reading Microscope Information from CZI image data.
2024-02-11 14:17:08,882 -  INFO - Reading SampleCarrier Information from CZI image data.
2024-02-11 14:17:08,883 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 14:17:08,884 -  INFO - No Scene or Well information found. Try to read XY Stage Coordinates from subblocks.
2024-02-11 14:17:09,118 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 14:17:09,126 -  INFO - Reading additional Metedata from CZI image data.
2

In [18]:
# write XML to disk
xmlfile = czimd.writexml(filepath)
print("XML File written to:", xmlfile)

XML File written to: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_CZI_MetaData.xml


In [19]:
# get the planetable for the CZI file
pt, csvfile = misc_tools.get_planetable(filepath,
                                  norm_time=True,
                                  savetable=True,
                                  separator=',',
                                  index=True)

# create a ipywdiget to show the dataframe with the metadata
wd2 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd2:
    display(HTML(pt.to_html()))
display(widgets.VBox(children=[wd2]))

print("PlaneTable CSV File written to:", csvfile)

2024-02-11 14:17:24,800 -  INFO - Reading Dimensions from CZI image data.
2024-02-11 14:17:25,318 -  INFO - PlaneTable saved as CSV file: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_planetable.csv


PlaneTable CSV File written to: E:\Github\czitools\data\CellDivision_T10_Z15_CH2_DCV_small_planetable.csv
